In [14]:
import os
from glob import glob
from pathlib import Path
import sys
import os

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn as sk
import pickle
import copy
from treeinterpreter import treeinterpreter as ti
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle

In [15]:
module_path = os.path.abspath(os.path.join('../../../'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [16]:
from global_config import ROOT_DIR

In [17]:
input_path = os.path.join(ROOT_DIR,'files/tests/preprocessing/dataset_creation/video_data_functionals_A220.npz')
output_path = os.path.join(ROOT_DIR,'files/out/functionals/supervised_learning/video')

In [18]:
npz_file = np.load(input_path, allow_pickle=True)
_XX = npz_file['x']
yy = npz_file['y']
col_names = npz_file['col_names']

scaler = MinMaxScaler()
XX = scaler.fit_transform(_XX)

In [19]:
XX, yy = shuffle(XX, yy)

In [20]:
from sklearn.ensemble import RandomForestClassifier

# Obtain X, y, and groups from the training dataset
X = pd.DataFrame(data=XX, columns=col_names)
y = pd.Series(y)


# TODO write code to create groups in src
groups = train_shuffled_df.group
n_groups = len(groups.unique())

# Load best parameters and initialize classifier
file_path = os.path.join(output_path,'best_params','best_params_rf_intensity_video.sav')
best_params = pickle.load(open(file_path, 'rb'))
clf = RandomForestClassifier(**best_params)

# Perform LOGO CV (5 GROUPS)
data_tmp = [] # Initialize to store data
logo = LeaveOneGroupOut()
for train_idx, val_idx in logo.split(X, y, groups):
    X_train, X_val, y_train, y_val = X.iloc[train_idx], X.iloc[val_idx], y.iloc[train_idx], y.iloc[val_idx]
    
    # Fit models
    clf.fit(X_train, y_train)
    
    # Step 0. Get prediction probabilities, bias, contributions, predicted class and store it in a DataFrame
    prediction_proba, bias, contributions = ti.predict(clf, X_val)
    prediction = clf.predict(X_val)
    for i in range(len(X_val)):
        pred_class_tmp = prediction[i]
        bias_tmp = bias[i, pred_class_tmp]
        contributions_tmp = contributions[i, :, pred_class_tmp]
        data_tmp.append([bias_tmp] + contributions_tmp.tolist() + [pred_class_tmp])

interpretation_step_0_df = pd.DataFrame(data=data_tmp, columns=['bias']+X.columns.tolist()+['prediction'])
interpretation_step_0_df

NameError: name 'train_shuffled_df' is not defined